In [1]:
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in xrange(steps):
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
                    for k in xrange(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = numpy.dot(P,Q)
        e = 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in xrange(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T


In [2]:
import numpy

In [11]:
R = [
         [5,3,0,1],
         [4,0,0,1],
         [1,1,0,5],
         [1,0,0,4],
         [0,1,5,4],
        ]

R = numpy.array(R)

N = len(R)
M = len(R[0])
K = 2

P = numpy.random.rand(N,K)
Q = numpy.random.rand(M,K)
%timeit
nP, nQ = matrix_factorization(R, P, Q, K)

In [12]:
print nP
print nQ

[[ 2.1841541  -0.30200044]
 [ 1.76716624 -0.14868645]
 [ 0.93856325  2.14919431]
 [ 0.83435726  1.69364753]
 [ 0.90208343  1.70481689]]
[[ 2.21043561 -0.49869745]
 [ 1.34804531 -0.12474915]
 [ 1.24353086  2.24706673]
 [ 0.73186193  1.98392035]]


In [13]:
nR = numpy.dot(nP, nQ.T)
print nR

[[ 4.97853886  2.98201299  2.03744788  0.99935442]
 [ 3.98035674  2.40076866  1.86341736  0.99833962]
 [ 1.0028359   0.99711561  5.99651539  4.95072906]
 [ 0.99967529  0.91347029  4.843288    3.97069611]
 [ 1.1438095   1.00337487  4.95260588  4.04242145]]
